In [26]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization,MaxPooling2D,Flatten,Dropout,Dense
from tensorflow.keras.applications.vgg16 import VGG16 as PretrainModel,preprocess_input
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from glob import glob
import os ,sys

In [2]:

cifar10=tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train,x_test=x_train/255,x_test/255
print(x_train.shape)

170498071/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)


In [4]:
#the data is 2d
#convolution expecst height x width x color
y_train=y_train.flatten()
y_test=y_test.flatten()
print(y_train.shape)

(50000,)


In [5]:
#the data is 2d
#convolution expecst height x width x color
y_train=y_train.flatten()
y_test=y_test.flatten()
print(y_train.shape)

(50000,)


In [6]:

k=len(set(y_train))
print(k)

10


In [19]:
ptm=PretrainModel(
    
                  input_shape=[32,32]+[3],
                  weights='imagenet',
                  include_top=False
)

58889256/58889256 [==============================] - 0s 0us/step


In [30]:
x=Flatten()(ptm.output)
x=Dense(k,activation='softmax')(x)
model=Model(inputs=ptm.input,outputs=x)

In [31]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [36]:
#data augmentation if have to 
batch_size=32
gen=tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,
                                                               rotation_range=0.2,horizontal_flip=False,vertical_flip=False)
train_generator=gen.flow(x_train,y_train,batch_size=batch_size)

valid_generator=gen.flow(x_test,y_test,batch_size=batch_size)
steps_per_epochs=x_train.shape[0//batch_size]

In [42]:
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r=model.fit_generator(train_generator,validation_data=valid_generator,epochs=8,
                      steps_per_epoch=steps_per_epochs)

In [ ]:
plt.plot(r.history['accuracy'],label="accuracy")
plt.plot(r.history['val_accuracy'],label='val_accuracy')

In [ ]:


plt.plot(r.history['loss'],label="loss")
plt.plot(r.history['val_loss'],label='val_loss')


In [ ]:
#show some misclassified example
misclassified_idx=np.where(p_test!=y_test)[0]
i=np.random.choice(misclassified_idx)
plt.imshow(x_test[i],cmap='gray')
plt.title("true label: %s predicted: %s"%(y_test[i],p_test[i]))
